# Reconnaissance des signalisations routières

<img src="https://saferautohome.files.wordpress.com/2019/04/traffic-signs-gtsdb.png?w=1400">

Il existe plusieurs datasets de "traffic signs", cf par exemple :  
https://saferauto.home.blog/2019/04/03/3o-databases-for-traffic-signs-detection/  
Ou :  
https://github.com/dctian/DeepPiCar/tree/master/models/object_detection/data/images/train  
(avec identification des zones pour de la détection d'objet)

On va utiliser ici un dataset Kaggle :  
https://www.kaggle.com/valentynsichkar/traffic-signs-preprocessed

## Importation des librairies

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AvgPool2D, BatchNormalization, Reshape

In [ ]:
# Lecture du répertoire Kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Lecture des données

Les données sont encodées avec Pickle :  
https://docs.python.org/3/library/pickle.html

In [ ]:
import pickle

On lit le fichier "data3.pickle" qui contient les images prétraitées

In [ ]:
with open('../input/traffic-signs-preprocessed/data3.pickle', 'rb') as f:
    data = pickle.load(f, encoding='latin1')

In [ ]:
X_train = data['x_train']
y_train = data['y_train']

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.transpose(0,2,3,1)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
y_train[0]

In [ ]:
from keras.utils.np_utils import to_categorical
y_cat = to_categorical(y_train)
y_cat[0]

In [ ]:
y_cat.shape

In [ ]:
labels = ['Speed limit (20km/h)',
 'Speed limit (30km/h)',
 'Speed limit (50km/h)',
 'Speed limit (60km/h)',
 'Speed limit (70km/h)',
 'Speed limit (80km/h)',
 'End of speed limit (80km/h)',
 'Speed limit (100km/h)',
 'Speed limit (120km/h)',
 'No passing',
 'No passing for vehicles over 3.5 metric tons',
 'Right-of-way at the next intersection',
 'Priority road',
 'Yield',
 'Stop',
 'No vehicles',
 'Vehicles over 3.5 metric tons prohibited',
 'No entry',
 'General caution',
 'Dangerous curve to the left',
 'Dangerous curve to the right',
 'Double curve',
 'Bumpy road',
 'Slippery road',
 'Road narrows on the right',
 'Road work',
 'Traffic signals',
 'Pedestrians',
 'Children crossing',
 'Bicycles crossing',
 'Beware of ice/snow',
 'Wild animals crossing',
 'End of all speed and passing limits',
 'Turn right ahead',
 'Turn left ahead',
 'Ahead only',
 'Go straight or right',
 'Go straight or left',
 'Keep right',
 'Keep left',
 'Roundabout mandatory',
 'End of no passing',
 'End of no passing by vehicles over 3.5 metric tons']

In [ ]:
plt.figure(figsize=(10,20))
for i in range(0,49) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    plt.imshow(X_train[i])
    plt.title(labels[y_train[i]])

In [ ]:
X_test = data['x_test'].transpose(0,2,3,1)
y_test = data['y_test']

In [ ]:
y_testc = to_categorical(y_test)

In [ ]:
y_testc.shape

In [ ]:
num_classes = len(labels)

In [ ]:
num_classes

## Exercice : utiliser les réseaux convolutionnels, et ensuite le transfer learning pour reconnaître les signalisations routières

In [ ]:
unique_elements, counts_elements = np.unique(y_test, return_counts=True)
print("Comptage des valeurs :")
print(np.asarray((unique_elements, counts_elements)))

In [ ]:
# Modèle CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(20, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(20, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(43))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
train = model.fit(X_train, y_cat, validation_data=(X_test, y_testc), epochs=30, batch_size=200, verbose=1)

In [ ]:
scores = model.evaluate(X_test, y_testc, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

Transfer Learing

In [ ]:
# Modèle CNN plus profond
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(20, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(43, activation='softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Apprentissage
train = model.fit(X_train, y_cat, validation_data=(X_test, y_testc), epochs=50, batch_size=200, verbose=1)



In [ ]:
scores = model.evaluate(X_test, y_testc, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))